# Supuestos en Regresión Lineal

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.stats.api as sms

In [ ]:
df = pd.read_csv(
    "D://Coderhouse//10. Estudios de Casos de Modelos Analíticos//statex77.csv"
)
df

In [ ]:
X = df.loc[
    :, ["Population", "Income", "Illiteracy", "Murder", "HS Grad", "Frost", "Area"]
]
X = sm.add_constant(X)
X

In [ ]:
Y = df["Life Exp"]
Y

In [ ]:
mod = sm.OLS(Y, X)
res = mod.fit()
print(res.summary())

In [ ]:
X_red = df.loc[:, ["Population", "Murder", "HS Grad", "Frost"]]
X_red = sm.add_constant(X_red)
X_red

In [ ]:
mod_red = sm.OLS(Y, X_red)
res_red = mod_red.fit()
print(res_red.summary())

## Verificación de supuestos

A continuación se verificará que el modelo cumpla los supuestos de la regresión lineal:

* Los errores tienen media cero.

* Los errores tienen la misma varianza \(σ^2\).

* Los errores son independientes entre sí.

* Los errores tienen una distribución Normal.

### Media cero
Para verificar este supuesto se realizará un diagrama de dispersión de los residuos frente a cada una de las variables independientes del modelo. Con estos diagramas se busca identificar si los residuos siguen algún tipo de comportamiento, por ejemplo, que todos los residuos tengan valores positivos o que aparentemente se distribuyan alrededor de una curva.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df["Murder"], residuos)
plt.plot(df["Murder"], [0] * len(df), color="red", linewidth=3)
plt.xlabel("Murder")
plt.ylabel("Residuos")
plt.show()

In [ ]:
residuos = res_red.resid
np.mean(residuos)

### Varianza constante
Para verificar el supuesto de varianza constante o homoscedasticidad se utilizarán dos métodos. El primero de ellos consiste en construir un diagramas de dispersión de los residuos contra los valores proyectados $(\hat{y}_i)$ de la siguiente forma:

In [ ]:
proyectado = res.fittedvalues
plt.scatter(proyectado, residuos)
plt.xlabel("Poyectados")
plt.ylabel("Residuos")
plt.show()

En este caso los residuos parecen distribuirse en una misma franja y por lo tanto no se encuentran indicios claros de heterocedasticidad. Sin embargo, este método es subjetivo por lo que se utilizará la prueba de Breusch Pagan para determinar si hay presencia de homocedasticidad. En la prueba de Breusch Pagan se tienen las siguientes hipótesis:

$H_0$: Hay homocedasticidad
$H_a$: Hay heterocedasticidad

In [ ]:
# Breusch - Pagan
name = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
test = sms.het_breuschpagan(residuos, res.model.exog)
pd.DataFrame(list(test), index=name, columns=["Breusch-Pagan"])

In [ ]:
# Goldfeld-Quandt
name = ["F statistic", "p-value", "type"]
test = sms.het_goldfeldquandt(residuos, res.model.exog)
pd.DataFrame(list(test), index=name, columns=["Goldfeld-Quant"])

### Normalidad
Para verificar el supuesto de normalidad es posible utilizar los gráficos cuartil-cuartil o Q-Q plot e histogramas:

In [ ]:
from scipy import stats

In [ ]:
sm.ProbPlot(res_red.resid).qqplot(line="s")
plt.title("Q-Q plot")

$H_0$: Los residuos siguen una distribución Normal

$H_a$: Los residuos no siguen una distribución Normal

In [ ]:
jb = stats.jarque_bera(res_red.resid)
sw = stats.shapiro(res_red.resid)
ad = stats.anderson(res_red.resid, dist="norm")
ks = stats.kstest(res_red.resid, "norm")

print(f"Jarque-Bera test ---- statistic: {jb[0]:.4f}, p-value: {jb[1]}")
print(f"Shapiro-Wilk test ---- statistic: {sw[0]:.4f}, p-value: {sw[1]:.4f}")
print(
    f"Kolmogorov-Smirnov test ---- statistic: {ks.statistic:.4f}, p-value: {ks.pvalue:.4f}"
)
print(
    f"Anderson-Darling test ---- statistic: {ad.statistic:.4f}, 5% critical value: {ad.critical_values[2]:.4f}"
)

Utilizando una significancia del 5% no se rechaza la hipótesis nula, por lo que se puede decir que el modelo cumple el supuesto de normalidad.

### Independencia
Para verificar el supuesto de independencia se utilizará la prueba de Durbin-Watson. Con esto se busca determinar si los errores se encuentran correlacionados entre sí o autocorrelacionados. Esta prueba utiliza las siguientes hipótesis:

$H_0$: No hay presencia de autocorrelación
$H_a$: Hay presencia de autocorrelación

In [ ]:
from statsmodels.stats.stattools import durbin_watson

durbinWatson = durbin_watson(residuos)
print("Valores de 1.5 < d < 2.5 normalmente muestran que no hay autocorrelacion")
print("0 a 2< hay autocorrelacion positiva")
print(">2 a 4 hay autocorrelacion negativa")
print("-------------------------------------")
print("Durbin-Watson:", durbinWatson)

In [ ]:
import statsmodels.tsa.api as smt

acf = smt.graphics.plot_acf(res_red.resid, lags=40, alpha=0.05)
acf.show()

### No multicolinealidad
Por último se realizará una prueba adicional para verificar que no exista presencia de multicolinealidad. Para esto se calcula el factor de inflación de la varianza (VIF) para cada una de las variables independientes del modelo, con el que se dirá que existe alta multicolinealidad cuando el VIF es mayor a 10.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor(X_red.values, i) for i in range(X_red.shape[1])]
pd.DataFrame(
    {"vif": vif[1:]},
    index=df.loc[:, ["Population", "Murder", "HS Grad", "Frost"]].columns,
).T

### Ejercicio Propuesto Práctica

**Utilice el conjunto de datos Ejercicio_Supuestos en la carpeta**

La revista Biometry publicó en el año de 1981 los siguientes datos que relacionan la contaminación del aire observada en 41 ciudades de los Estados Unidos con un conjunto de seis variables explicativas definidas en los términos siguientes:

y: concentración promedio anual de dióxido de azufre (microgramos por metro cúbico)
x1: temperatura promedio anual (ºF)
x2: número de fábricas que emplean 20 o más trabajadores

x3: número de habitantes (en miles) según el censo de 1970
x4: velocidad promedio anual del viento (millas por hora)
x5: precipitación promedio anual (pulgadas)
x6: número promedio de días con lluvias al año

Se desea construir un modelo que permita caracterizar la relación entre la contaminación del aire y las seis variables explicativas, como también predecir el nivel de contaminación que se obtendría para ciertos valores de las variables explicativas previamente definidos.

1.	Grafique la variable y vs. cada una de las variables explicativas. A partir de los gráficos, determine si se requieren términos de orden superior para representar en el modelo alguna de las variables explicativas.
2.	¿Existe alguna evidencia de multicolinealidad entre las variables explicativas del modelo? Utilice en su diagnóstico:
    a.	Diagramas de dispersión
    b.	Matriz de correlaciones
3.	Calcule el factor de inflación de varianza para cada una de las variables explicativas de un modelo lineal que relaciona y con las variables x1 a x6. A partir de estos factores, ¿considera usted que hay un problema de multicolinealidad entre las variables explicativas? 
4.	¿Cuáles variables explican en mayor grado la contaminación del aire?
5.	Realice una reducción de variables, luego con el modelo reducido responda:
    a.	¿ los residuos siguen una distribución normal? Justifique su respuesta.
    b.	¿ se satisface el supuesto de varianza constante? Justifique su respuesta.
6.	Estime la concentración promedio anual de dióxido de azufre en el aire en una ciudad que reporta los siguientes valores para las seis variables explicativas: x1 = 60, x2 = 150, x3 = 600, x4 = 10, x5 = 40, x6 = 100